# Aplicación web de análisis de sentimientos

En este cuaderno usaremos el servicio SageMaker de Amazon para construir un modelo de árbol aleatorio para predecir el sentimiento de una reseña de película. Además, implementaremos este modelo en un punto final y construiremos una aplicación web muy simple que interactuará con el punto final implementado de nuestro modelo.

## Bosquejo general
Por lo general, cuando use una instancia de notebook con SageMaker, procederá con los siguientes pasos. Por supuesto, no todos los pasos deberán realizarse con cada proyecto. Además, hay mucho margen de variación en muchos de los pasos, como verá a lo largo de estas lecciones.

* Descargue o recupere los datos.
* Procesar / preparar los datos.
* Cargue los datos procesados a S3.
* Entrenar a un modelo elegido.
* Probar el modelo entrenado (generalmente usando un trabajo de transformación por lotes).
* Implementar el modelo entrenado.
* Use el modelo desplegado.

En este cuaderno avanzaremos a través de cada uno de los pasos anteriores. También veremos que el paso final, usar el modelo implementado, puede ser bastante desafiante.

## Paso 1: Descargar los datos
El conjunto de datos que vamos a utilizar es muy popular entre los investigadores del procesamiento del lenguaje natural, generalmente denominado conjunto de datos IMDb. Consiste en reseñas de películas del sitio web imdb.com, cada una etiquetada como 'positiva' si el crítico disfrutó la película, o 'negativa' de lo contrario.

In [ ]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

## Paso 2: preparación y procesamiento de los datos
Los datos que hemos descargado se dividen en varios archivos, cada uno de los cuales contiene una única revisión. Será mucho más fácil avanzar si combinamos estos archivos individuales en dos archivos grandes, uno para capacitación y otro para prueba.

In [ ]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [ ]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

In [ ]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [ ]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

In [ ]:
train_X[100]

## Procesando los datos
Ahora que tenemos nuestros conjuntos de datos de entrenamiento y prueba combinados y listos para usar, necesitamos comenzar a procesar los datos sin procesar en algo que nuestro algoritmo de aprendizaje automático pueda usar. Para comenzar, eliminamos cualquier formato html y cualquier carácter no alfanumérico que pueda aparecer en las revisiones. Haremos esto de una manera muy simple usando el módulo de expresión regular de Python. Discutiremos la razón de este preprocesamiento bastante simplista más adelante.

In [ ]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def review_to_words(review):
    words = REPLACE_NO_SPACE.sub("", review.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    return words

In [ ]:
review_to_words(train_X[100])

In [ ]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_web_app")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [ ]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

## Extraer características de la bolsa de palabras
Para el modelo que implementaremos, en lugar de utilizar las revisiones directamente, vamos a transformar cada revisión en una representación característica de la Bolsa de palabras. Tenga en cuenta que 'en la naturaleza' solo tendremos acceso al conjunto de entrenamiento para que nuestro transformador solo pueda usar el conjunto de entrenamiento para construir una representación.

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
# joblib is an enhanced version of pickle that is more efficient for storing NumPy arrays

def extract_BoW_features(words_train, words_test, vocabulary_size=5000,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""
    
    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Fit a vectorizer to training documents and use it to transform them
        # NOTE: Training documents have already been preprocessed and tokenized into words;
        #       pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x
        vectorizer = CountVectorizer(max_features=vocabulary_size)
        features_train = vectorizer.fit_transform(words_train).toarray()

        # Apply the same vectorizer to transform the test documents (ignore unknown words)
        features_test = vectorizer.transform(words_test).toarray()
        
        # NOTE: Remember to convert the features using .toarray() for a compact representation
        
        # Write to cache file for future runs (store vocabulary as well)
        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        features_train, features_test, vocabulary = (cache_data['features_train'],
                cache_data['features_test'], cache_data['vocabulary'])
    
    # Return both the extracted features as well as the vocabulary
    return features_train, features_test, vocabulary

In [ ]:
train_X, test_X, vocabulary = extract_BoW_features(train_X, test_X)

In [ ]:
len(train_X[100])

## Paso 3: subir datos a S3
Ahora que hemos creado la representación de características de nuestros datos de entrenamiento (y pruebas), es hora de comenzar a configurar y usar el clasificador XGBoost provisto por SageMaker.

## Escribiendo los conjuntos de datos
El clasificador XGBoost que utilizaremos requiere que el conjunto de datos se escriba en un archivo y se almacene con Amazon S3. Para hacer esto, comenzaremos dividiendo el conjunto de datos de entrenamiento en dos partes, los datos con los que entrenaremos al modelo y un conjunto de validación. Luego, escribiremos esos conjuntos de datos en un archivo localmente y luego cargaremos los archivos en S3. Además, escribiremos el conjunto de prueba en un archivo y lo cargaremos en S3. Esto es para que podamos usar la funcionalidad de Transformación por lotes de SageMakers para probar nuestro modelo una vez que lo hayamos ajustado.

In [ ]:
import pandas as pd

# Earlier we shuffled the training dataset so to make things simple we can just assign
# the first 10 000 reviews to the validation set and use the remaining reviews for training.
val_X = pd.DataFrame(train_X[:10000])
train_X = pd.DataFrame(train_X[10000:])

val_y = pd.DataFrame(train_y[:10000])
train_y = pd.DataFrame(train_y[10000:])

In [ ]:
data_dir = '../data/sentiment_web_app'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
pd.DataFrame(test_X).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)

pd.concat([val_y, val_X], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)
pd.concat([train_y, train_X], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [ ]:
# To save a bit of memory we can set text_X, train_X, val_X, train_y and val_y to None.
test_X = train_X = val_X = train_y = val_y = None

## Carga de archivos de entrenamiento / validación a S3
El servicio S3 de Amazon nos permite almacenar archivos a los que pueden acceder tanto los modelos de entrenamiento incorporados como el modelo XGBoost que usaremos como los modelos personalizados como el que veremos un poco más adelante.

Para esta y la mayoría de las otras tareas que realizaremos con SageMaker, hay dos métodos que podríamos usar. El primero es utilizar la funcionalidad de bajo nivel de SageMaker que requiere conocer cada uno de los objetos involucrados en el entorno de SageMaker. El segundo es utilizar la funcionalidad de alto nivel en la que se han hecho ciertas elecciones en nombre del usuario. El enfoque de bajo nivel se beneficia al permitir al usuario una gran flexibilidad mientras que el enfoque de alto nivel hace que el desarrollo sea mucho más rápido. Para nuestros propósitos, optaremos por utilizar el enfoque de alto nivel, aunque usar el enfoque de bajo nivel es ciertamente una opción.

Recuerde el método upload_data () que es un miembro del objeto que representa nuestra sesión actual de SageMaker. Lo que hace este método es cargar los datos en el depósito predeterminado (que se crea si no existe) en la ruta descrita por la variable key_prefix. Para ver esto por sí mismo, una vez que haya cargado los archivos de datos, vaya a la consola S3 y mire para ver dónde se han cargado los archivos.

In [ ]:
import sagemaker

session = sagemaker.Session() # Store the current SageMaker session

# S3 prefix (which folder will we use)
prefix = 'sentiment-web-app'

test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

## Paso 4: Crear el modelo XGBoost
Ahora que los datos se han cargado, es hora de crear el modelo XGBoost. Para empezar, necesitamos hacer algo de configuración. En este punto, vale la pena discutir qué es un modelo en SageMaker. Es más fácil pensar en un modelo que comprenda tres objetos diferentes en el ecosistema SageMaker, que interactúan entre sí.

* Artefactos modelo
* Código de entrenamiento (contenedor)
* Código de inferencia (contenedor)

Los artefactos del modelo son lo que usted podría considerar como el modelo en sí. Por ejemplo, si estuviera construyendo una red neuronal, los artefactos del modelo serían los pesos de las diversas capas. En nuestro caso, para un modelo XGBoost, los artefactos son los árboles reales que se crean durante el entrenamiento.

Los otros dos objetos, el código de entrenamiento y el código de inferencia se utilizan para manipular los artefactos de entrenamiento. Más precisamente, el código de entrenamiento usa los datos de entrenamiento que se proporcionan y crea los artefactos del modelo, mientras que el código de inferencia usa los artefactos del modelo para hacer predicciones sobre nuevos datos.

La forma en que SageMaker ejecuta el código de capacitación e inferencia es mediante el uso de contenedores Docker. Por ahora, piense en un contenedor como una forma de empaquetar código para que las dependencias no sean un problema.

In [ ]:
from sagemaker import get_execution_role

# Our current execution role is required when creating the model as the training
# and inference code will need to access the model artifacts.
role = get_execution_role()

In [ ]:
# We need to retrieve the location of the container which is provided by Amazon for using XGBoost.
# As a matter of convenience, the training and inference code both use the same container.
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(session.boto_region_name, 'xgboost', 'latest')

In [ ]:
# First we create a SageMaker estimator object for our model.
xgb = sagemaker.estimator.Estimator(container, # The location of the container we wish to use
                                    role,                                    # What is our current IAM Role
                                    train_instance_count=1,                  # How many compute instances
                                    train_instance_type='ml.m4.xlarge',      # What kind of compute instances
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                    sagemaker_session=session)

# And then set the algorithm specific parameters.
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=500)

## Ajustar el modelo XGBoost
Ahora que nuestro modelo ha sido configurado, simplemente necesitamos adjuntar los conjuntos de datos de capacitación y validación y luego pedirle a SageMaker que configure el cálculo.

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

In [ ]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

## Paso 5: Prueba del modelo
Ahora que nos hemos adaptado a nuestro modelo XGBoost, es hora de ver qué tan bien funciona. Para hacerlo, utilizaremos la funcionalidad de transformación por lotes de SageMakers. Batch Transform es una forma conveniente de realizar inferencia en un gran conjunto de datos de una manera que no es en tiempo real. Es decir, no necesariamente necesitamos usar los resultados de nuestro modelo inmediatamente y en su lugar podemos realizar inferencias en una gran cantidad de muestras. Un ejemplo de esto en la industria podría ser realizar un informe de fin de mes. Este método de inferencia también puede ser útil para nosotros, ya que significa que podemos realizar inferencia en todo nuestro conjunto de pruebas.

Para realizar una transformación por lotes, primero debemos crear un transformador a partir de nuestro objeto estimador entrenado.

In [ ]:
xgb_transformer = xgb.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')

A continuación, realizamos el trabajo de transformación. Al hacerlo, debemos asegurarnos de especificar el tipo de datos que estamos enviando para que se serialicen correctamente en segundo plano. En nuestro caso, proporcionamos a nuestro modelo datos csv, por lo que especificamos text / csv. Además, si los datos de prueba que proporcionamos son demasiado grandes para procesarlos a la vez, entonces debemos especificar cómo se debe dividir el archivo de datos. Como cada línea es una entrada única en nuestro conjunto de datos, le decimos a SageMaker que puede dividir la entrada en cada línea.

In [ ]:
xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')

Actualmente el trabajo de transformación se está ejecutando pero lo está haciendo en segundo plano. Como deseamos esperar hasta que se complete el trabajo de transformación y nos gustaría un poco de retroalimentación, podemos ejecutar el método wait().

In [ ]:
xgb_transformer.wait()

Ahora el trabajo de transformación se ha ejecutado y el resultado, el sentimiento estimado de cada revisión, se ha guardado en S3. Como preferimos trabajar en este archivo localmente, podemos realizar un poco de magia de cuaderno para copiar el archivo en el data_dir.

In [ ]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

El último paso ahora es leer la salida de nuestro modelo, convertir la salida a algo un poco más utilizable, en este caso queremos que el sentimiento sea 1 (positivo) o 0 (negativo), y luego compararlo con el suelo Etiquetas de verdad.

In [ ]:
predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

## Paso 6: Implementación del modelo
Una vez que construimos y ajustamos nuestro modelo, SageMaker almacena los artefactos del modelo resultante y podemos usarlos para implementar un punto final (código de inferencia). Para ver esto, mire en la consola de SageMaker y debería ver que se ha creado un modelo junto con un enlace a la ubicación S3 donde se han almacenado los artefactos del modelo.

Implementar un punto final es muy similar a entrenar el modelo con algunas diferencias importantes. El primero es que un modelo implementado no cambia los artefactos del modelo, por lo que al enviarlo a varias instancias de prueba, el modelo no cambiará. Otra diferencia es que, dado que no estamos realizando un cálculo fijo, como estábamos en el paso de entrenamiento o mientras realizamos una transformación por lotes, la instancia de proceso que se inicia permanece en funcionamiento hasta que le decimos que se detenga. Es importante tener en cuenta que si lo olvidamos y lo dejamos en funcionamiento, se nos cobrará todo el tiempo.

En otras palabras **¡Si ya no estás usando un punto final desplegado, apágalo!**

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

## Prueba del modelo (nuevamente)
Ahora que hemos implementado nuestro punto final, podemos enviarle los datos de prueba y recuperar los resultados de la inferencia. Ya lo hicimos anteriormente usando la funcionalidad de transformación por lotes de SageMaker, sin embargo, probaremos nuestro modelo nuevamente usando el punto final recientemente implementado para asegurarnos de que funcione correctamente y tener una idea de cómo funciona el punto final.

Cuando se utiliza el punto final creado, es importante saber que tenemos una cantidad limitada de información que podemos enviar en cada llamada, por lo que debemos dividir los datos de prueba en fragmentos y luego enviar cada fragmento. Además, debemos serializar nuestros datos antes de enviarlos al punto final para garantizar que nuestros datos se transmitan correctamente. Afortunadamente, SageMaker puede hacer la parte de serialización por nosotros siempre que le informemos el formato de nuestros datos.

In [ ]:
from sagemaker.predictor import csv_serializer

# We need to tell the endpoint what format the data we are sending is in so that SageMaker can perform the serialization.
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

In [ ]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])
    
    return np.fromstring(predictions[1:], sep=',')

In [ ]:
test_X = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None).values

predictions = predict(test_X)
predictions = [round(num) for num in predictions]

Por último, verificamos cuál es la precisión de nuestro modelo.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

Y los resultados aquí deberían estar de acuerdo con la prueba del modelo que hicimos anteriormente usando el trabajo de transformación por lotes.

## Limpiar
Ahora que hemos determinado que la implementación de nuestro modelo funciona como se esperaba, vamos a cerrarlo. Recuerde que cuanto más tiempo se ejecute el punto final, mayor será el costo y dado que tenemos un poco más de trabajo por hacer antes de que podamos usar nuestro punto final con nuestra aplicación web simple, debemos cerrar todo.

In [ ]:
xgb_predictor.delete_endpoint()

## Paso 7: Poner nuestro modelo a trabajar
Como hemos mencionado algunas veces, nuestro objetivo es implementar nuestro modelo y luego acceder a él utilizando una aplicación web muy simple. La intención es que esta aplicación web tome algunos datos enviados por el usuario (una revisión), los envíe a nuestro punto final (el modelo) y luego muestre el resultado.

Sin embargo, hay una pequeña trampa. Actualmente, la única forma en que podemos acceder al punto final para enviar datos es mediante la API de SageMaker. Podemos, si lo deseamos, exponer la URL real de la que el punto final de nuestro modelo recibe datos, sin embargo, si solo la enviamos nosotros mismos, no obtendremos nada a cambio. Esto se debe a que el punto final creado por SageMaker requiere que la entidad que accede tenga los permisos correctos. Por lo tanto, tendríamos que autenticar de alguna manera nuestra aplicación web con AWS.

Tener un sitio web que se autentique en AWS parece un poco más allá del alcance de esta lección, por lo que optaremos por un enfoque alternativo. Es decir, crearemos un nuevo punto final que no requiera autenticación y que actúe como un proxy para el punto final SageMaker.

Como restricción adicional, intentaremos evitar el procesamiento de datos en la propia aplicación web. Recuerde que cuando construimos y probamos nuestro modelo, comenzamos con una revisión de la película, luego lo simplificamos eliminando cualquier formato y puntuación html, luego construimos una bolsa de palabras incrustadas y el vector resultante es lo que enviamos a nuestro modelo. Todo esto también debe hacerse a nuestra entrada del usuario.

Afortunadamente, podemos hacer todo este procesamiento de datos en el backend, utilizando el servicio Lambda de Amazon.

El diagrama anterior brinda una descripción general de cómo los diversos servicios funcionarán juntos. En el extremo derecho está el modelo que hemos entrenado anteriormente y que se implementará con SageMaker. En el extremo izquierdo está nuestra aplicación web que recopila la crítica de la película de un usuario, la envía y espera un sentimiento positivo o negativo a cambio.

En el medio es donde sucede algo de la magia. Construiremos una función Lambda, que se puede considerar como una función simple de Python que se puede ejecutar siempre que ocurra un evento específico. Esta función de Python realizará el procesamiento de datos que necesitamos realizar en una revisión enviada por el usuario. Además, le daremos permiso a esta función para enviar y recibir datos desde un punto final de SageMaker.

Por último, el método que usaremos para ejecutar la función Lambda es un nuevo punto final que crearemos usando API Gateway. Este punto final será una url que escuchará los datos que se le enviarán. Una vez que obtiene algunos datos, los pasará a la función Lambda y luego devolverá lo que devuelva la función Lambda. Básicamente, actuará como una interfaz que permite que nuestra aplicación web se comunique con la función Lambda.

## Procesando una sola revisión
Por ahora, supongamos que nuestro usuario nos da una reseña de la película en forma de cadena, de esta manera:

In [ ]:
test_review = "Nothing but a disgusting materialistic pageant of glistening abed remote control greed zombies, totally devoid of any heart or heat. A romantic comedy that has zero romantic chemestry and zero laughs!"

¿Cómo pasamos de esta cadena al vector de características de bolsa de palabras que espera nuestro modelo?

Si recordamos al principio de este cuaderno, el primer paso es eliminar los caracteres innecesarios utilizando el método review_to_words. Recuerde que lo hicimos intencionalmente de una manera muy simplista. Esto se debe a que vamos a tener que copiar este método en nuestra (eventual) función Lambda (entraremos en más detalles más adelante) y esto significa que debe ser bastante simplista.

In [ ]:
test_words = review_to_words(test_review)
print(test_words)

A continuación, necesitamos construir una bolsa de palabras incrustadas en la cadena test_words. Para hacer esto, recuerde que la inclusión de una bolsa de palabras utiliza un vocabulario que consiste en las palabras que aparecen con más frecuencia en un conjunto de documentos. Luego, para cada palabra en el vocabulario, registramos la cantidad de veces que esa palabra aparece en test_words. Construimos el vocabulario anteriormente usando el conjunto de entrenamiento para nuestro problema, por lo que codificar test_words es relativamente sencillo.

In [ ]:
def bow_encoding(words, vocabulary):
    bow = [0] * len(vocabulary) # Start by setting the count for each word in the vocabulary to zero.
    for word in words.split():  # For each word in the string
        if word in vocabulary:  # If the word is one that occurs in the vocabulary, increase its count.
            bow[vocabulary[word]] += 1
    return bow

In [ ]:
test_bow = bow_encoding(test_words, vocabulary)
print(test_bow)

In [ ]:
len(test_bow)

Entonces, ahora sabemos cómo construir una bolsa de palabras que codifique una revisión proporcionada por el usuario, ¿cómo enviarla a nuestro punto final? Primero, tenemos que volver a iniciar el punto final.

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

En este punto, podríamos hacer lo mismo que hicimos antes cuando probamos nuestro modelo desplegado y enviamos test_bow a nuestro punto final usando el objeto xgb_predictor. Sin embargo, cuando finalmente construyamos nuestra función Lambda, no tendremos acceso a este objeto, entonces, ¿cómo llamamos a un punto final SageMaker?

Resulta que las funciones de Python que se usan en Lambda tienen acceso a otra biblioteca de Amazon llamada boto3. Esta biblioteca proporciona una API para trabajar con los servicios de Amazon, incluido SageMaker. Para empezar, necesitamos controlar el tiempo de ejecución de SageMaker.

In [ ]:
import boto3

runtime = boto3.Session().client('sagemaker-runtime')

Y ahora que tenemos acceso al tiempo de ejecución de SageMaker, podemos pedirle que utilice (invoque) un punto final que ya se ha creado. Sin embargo, debemos proporcionar a SageMaker el nombre del punto final desplegado. Para descubrir esto, podemos imprimirlo usando el objeto xgb_predictor.

In [ ]:
xgb_predictor.endpoint

Usando el tiempo de ejecución de SageMaker y el nombre de nuestro punto final, podemos invocar el punto final y enviarle los datos test_bow.

In [ ]:
response = runtime.invoke_endpoint(EndpointName = xgb_predictor.endpoint, # The name of the endpoint we created
                                   ContentType = 'text/csv',                     # The data format that is expected
                                   Body = test_bow)

Entonces, ¿por qué recibimos un error?

Porque tratamos de enviar al punto final una lista de enteros, pero esperaba que enviáramos datos de tipo text / csv. Entonces, necesitamos convertirlo.

In [ ]:
response = runtime.invoke_endpoint(EndpointName = xgb_predictor.endpoint, # The name of the endpoint we created
                                   ContentType = 'text/csv',                     # The data format that is expected
                                   Body = ','.join([str(val) for val in test_bow]).encode('utf-8'))
print(response)

Como podemos ver, la respuesta de nuestro modelo es un dict de aspecto algo complicado que contiene mucha información. Lo que más nos interesa es el objeto 'Cuerpo', que es un objeto de transmisión que necesitamos leer para poder usarlo.

In [ ]:
response = response['Body'].read().decode('utf-8')
print(response)

Ahora que sabemos cómo procesar los datos entrantes del usuario, podemos comenzar a configurar la infraestructura para que nuestra aplicación web simple funcione. Para hacer esto, haremos uso de dos servicios diferentes. Servicios de Amazon Lambda y API Gateway.

Lambda es un servicio que permite a alguien escribir un código relativamente simple y ejecutarlo cada vez que ocurre un disparador elegido. Por ejemplo, es posible que desee actualizar una base de datos cada vez que se carguen nuevos datos en una carpeta almacenada en S3.

API Gateway es un servicio que le permite crear puntos finales HTTP (direcciones URL) que están conectados a otros servicios de AWS. Uno de los beneficios de esto es que puede decidir qué credenciales, si corresponde, son necesarias para acceder a estos puntos finales.

En nuestro caso, vamos a configurar un punto final HTTP a través de API Gateway que está abierto al público. Luego, cada vez que alguien envíe datos a nuestro punto final público, activaremos una función Lambda que enviará la entrada (en nuestro caso, una revisión) al punto final de nuestro modelo y luego devolverá el resultado.

Configurar una función Lambda
Lo primero que vamos a hacer es configurar una función Lambda. Esta función Lambda se ejecutará siempre que nuestra API pública tenga datos enviados. Cuando se ejecute, recibirá los datos, realizará cualquier tipo de procesamiento que sea necesario, enviará los datos (la revisión) al punto final de SageMaker que hemos creado y luego devolverá el resultado.

Parte A: Crear un rol de IAM para la función Lambda
Como queremos que la función Lambda llame a un punto final de SageMaker, debemos asegurarnos de que tenga permiso para hacerlo. Para hacer esto, construiremos un rol que luego podemos asignarle a la función Lambda.

Con la consola de AWS, navegue a la página de IAM y haga clic en Roles. Luego, haga clic en Crear rol. Asegúrese de que el servicio de AWS sea el tipo de entidad de confianza seleccionada y elija Lambda como el servicio que utilizará esta función, luego haga clic en Siguiente: Permisos.

En el cuadro de búsqueda, escriba sagemaker y seleccione la casilla de verificación junto a la política AmazonSageMakerFullAccess. Luego, haga clic en Siguiente: Revisar.

Por último, dale un nombre a este rol. Asegúrese de utilizar un nombre que recordará más adelante, por ejemplo, LambdaSageMakerRole. Luego, haga clic en Crear rol.

Parte B: crear una función Lambda
Ahora es el momento de crear realmente la función Lambda. Recuerde que antes para procesar la entrada proporcionada por el usuario y enviarla a nuestro punto final, necesitamos recopilar dos datos:

* El nombre del punto final y
* El objeto de vocabulario.
Copiaremos estos datos a nuestra función Lambda después de crearla.

Para comenzar, usando la consola de AWS, navegue a la página de AWS Lambda y haga clic en Crear una función. Cuando llegue a la página siguiente, asegúrese de que esté seleccionado Autor desde cero. Ahora, nombre su función Lambda, usando un nombre que recordará más adelante, por ejemplo sentiment_analysis_xgboost_func. Asegúrese de que el tiempo de ejecución de Python 3.6 esté seleccionado y luego elija el rol que creó en la parte anterior. Luego, haga clic en Crear función.

En la página siguiente verá información sobre la función Lambda que acaba de crear. Si se desplaza hacia abajo, debería ver un editor en el que puede escribir el código que se ejecutará cuando se active su función Lambda. Al recopilar el código que escribimos anteriormente para procesar una única revisión y agregarlo al ejemplo lambda_handler proporcionado, llegamos a lo siguiente.

Image

Una vez que haya copiado y pegado el código anterior en el editor de código Lambda, reemplace la porción **ENDPOINT NAME HERE** con el nombre del punto final que implementamos anteriormente. Puede determinar el nombre del punto final utilizando la celda de código a continuación.

In [ ]:
xgb_predictor.endpoint

Además, deberá copiar el diccionario de vocabulario en el lugar apropiado en el código al comienzo del método lambda_handler. La celda a continuación imprime el diccionario de vocabulario de una manera fácil de copiar y pegar.

In [ ]:
print(str(vocabulary))

Una vez que haya agregado el nombre del punto final a la función Lambda, haga clic en **Guardar**. Su función Lambda ahora está en funcionamiento. A continuación, debemos crear una forma para que nuestra aplicación web ejecute la función Lambda.

## Configuración de API Gateway
Ahora que nuestra función Lambda está configurada, es hora de crear una nueva API utilizando API Gateway que activará la función Lambda que acabamos de crear.

Con la consola de AWS, navegue hasta **Amazon API Gateway** y luego haga clic en **Comenzar**.

En la página siguiente, asegúrese de que **Nueva API** esté seleccionada y asigne un nombre a la nueva API, por ejemplo, sentiment_analysis_web_app. Luego, haga clic en **Crear API**.

Ahora hemos creado una API, sin embargo, actualmente no hace nada. Lo que queremos que haga es activar la función Lambda que creamos anteriormente.

Seleccione el menú desplegable **Acciones** y haga clic en **Crear método**. Se creará un nuevo método en blanco, seleccione su menú desplegable y seleccione **POST**, luego haga clic en la marca de verificación junto a él.

Para el punto de integración, asegúrese de que **Función Lambda** esté seleccionada y haga clic en **Usar integración de proxy Lambda**. Esta opción asegura que los datos que se envían a la API se envían directamente a la función Lambda sin procesamiento. También significa que el valor de retorno debe ser un objeto de respuesta adecuado, ya que API Gateway tampoco lo procesará.

Escriba el nombre de la **función Lambda** que creó anteriormente en el cuadro de entrada de texto de la función Lambda y luego haga clic en **Guardar**. Haga clic en **Aceptar** en el cuadro emergente que luego aparece, dando permiso a API Gateway para invocar la función Lambda que creó.

El último paso para crear API Gateway es seleccionar el menú desplegable **Acciones** y hacer clic en **Implementar API**. Deberá crear una nueva etapa de implementación y asignarle el nombre que desee, por ejemplo, prod.

Ahora ha configurado con éxito una API pública para acceder a su modelo SageMaker. Asegúrese de copiar o anotar la URL proporcionada para invocar su API pública recién creada, ya que será necesaria en el siguiente paso. Esta URL se puede encontrar en la parte superior de la página, resaltada en azul junto al texto **Invocar URL**.

## Paso 7: Implementar nuestra aplicación web
Ahora que tenemos una API disponible públicamente, podemos comenzar a usarla en una aplicación web. Para nuestros propósitos, hemos proporcionado un archivo html estático simple que puede hacer uso de la API pública que creó anteriormente.

En la carpeta del sitio web debe haber un archivo llamado index.html. Descargue el archivo en su computadora y ábralo en el editor de texto que elija. Debe haber una línea que contenga **REPLACE WITH PUBLIC API URL**. Reemplace esta cadena con la url que anotó en el último paso y luego guarde el archivo.

Ahora, si abre index.html en su computadora local, su navegador se comportará como un servidor web local y podrá usar el sitio proporcionado para interactuar con su modelo SageMaker.

Si desea ir más allá, puede alojar este archivo html en cualquier lugar que desee, por ejemplo, usando github o alojando un sitio estático en el S3 de Amazon. Una vez que hayas hecho esto, puedes compartir el enlace con cualquier persona que quieras y que también jueguen con él.

**Nota importante** Para que la aplicación web se comunique con el punto final de SageMaker, el punto final debe implementarse y ejecutarse. Esto significa que está pagando por ello. Asegúrese de que el punto final se esté ejecutando cuando desee usar la aplicación web, pero que lo cierre cuando no lo necesite, de lo contrario terminará con una factura de AWS sorprendentemente grande.

## Eliminar el punto final
Recuerde siempre cerrar su punto final si ya no lo está usando. Se le cobra por el período de tiempo que el punto final se está ejecutando, por lo que si lo olvida y lo deja encendido, podría terminar con una factura inesperadamente grande.

In [ ]:
xgb_predictor.delete_endpoint()

## Opcional: limpiar
La instancia de notebook predeterminada en SageMaker no tiene mucho espacio en disco disponible. A medida que continúe completando y ejecutando cuadernos, eventualmente llenará este espacio en disco, lo que generará errores que pueden ser difíciles de diagnosticar. Una vez que haya terminado de usar un cuaderno, es una buena idea eliminar los archivos que creó en el camino. Por supuesto, puede hacerlo desde la terminal o desde el hub del portátil si lo desea. La celda a continuación contiene algunos comandos para limpiar los archivos creados desde el cuaderno.

In [ ]:
# First we will remove all of the files contained in the data_dir directory
!rm $data_dir/*

# And then we delete the directory itself
!rmdir $data_dir

# Similarly we remove the files in the cache_dir directory and the directory itself
!rm $cache_dir/*
!rmdir $cache_dir